In [64]:
import pandas as pd
import numpy as np
import random
import networkx as nx
from tqdm import tqdm
import re
import matplotlib.pyplot as plt
import itertools
import csv

from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split

from sklearn.model_selection import RandomizedSearchCV
from time import time
from xgboost import XGBClassifier

from node2vec import Node2Vec
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import optuna
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

# Counting all nodes

In [65]:
nodes = dict()

with open("train-mod.txt") as file :
    end = file.seek(0, 2)
    file.seek(0)
    while file.tell() != end:
        line = file.readline().split()
        edges = list(itertools.combinations(line,2))
        for i in edges:
            if nodes.get(i) == None:
                node1 = i[0]
                node2 = i[1]
                if nodes.get((node2,node1)) == None:
                    nodes[i] = 1
                else:
                    nodes[(node2,node1)] += 1
            else:
                nodes[i] +=1

print(len(nodes))

16087


In [66]:
with open("weighted_graph.csv", "w", newline="") as a_file:

    writer = csv.writer(a_file)
    for key, value in nodes.items():
        writer.writerow([key[0], key[1], value])

    a_file.close()

In [67]:
g = nx.read_weighted_edgelist('weighted_graph.csv', delimiter=',', nodetype=int)

In [68]:
print(nx.info(g))

Name: 
Type: Graph
Number of nodes: 3816
Number of edges: 16087
Average degree:   8.4313
None


# Positive edges

In [70]:
edges_pos = list(nodes.keys())

In [71]:
with open("edges_pos_all.csv","w",newline="") as csvfile:
    writer=csv.writer(csvfile)
    writer.writerow(["Source","Target", "Label"])
    for edge in edges_pos:
        writer.writerow([edge[0], edge[1], 1])

# Generating negative edges (random sampling)

In [72]:
i = 0
num_test_edges = 16087
edges_neg = []
while i < num_test_edges:
    edge = random.sample(g.nodes(), 2)
    try:
        edge_exists = g.has_edge(edge[0],edge[1])
        if edge_exists == False:
            edges_neg.append([edge[0],edge[1]])
            i = i+1
    except Exception as e:
        pass

In [73]:
with open("edges_neg_16k.csv","w",newline="") as csvfile:
    writer=csv.writer(csvfile)
    writer.writerow(["Source","Target", "Label"])
    for edge in edges_neg:
        writer.writerow([edge[0], edge[1], 0])

# Train/Test split

In [75]:
edges_positive = pd.read_csv('edges_pos_all.csv').to_numpy()
edges_negative = pd.read_csv('edges_neg_16k.csv').to_numpy()

In [78]:
#reading total data df
df_pos = pd.DataFrame(edges_positive, columns=['source_node', 'destination_node', 'label'])
df_neg = pd.DataFrame(edges_negative, columns=['source_node', 'destination_node', 'label'])

In [81]:
df_pos_n2v =  df_pos[['source_node', 'destination_node']].copy()

In [83]:
# Generate walks
node2vec = Node2Vec(g, dimensions=30, walk_length=16, num_walks=50)

# train node2vec model
n2w_model = node2vec.fit(window=10, min_count=1)

Generating walks (CPU: 1): 100%|██████████| 50/50 [01:07<00:00,  1.36s/it]


In [84]:
data = pd.concat([df_pos, df_neg], ignore_index=True)

In [85]:
x_train_n2v = [(n2w_model[str(i)]+n2w_model[str(j)]) for i,j in zip(data['source_node'], data['destination_node'])]

<ipython-input-85-6d09be66ec01>:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  x_train_n2v = [(n2w_model[str(i)]+n2w_model[str(j)]) for i,j in zip(data['source_node'], data['destination_node'])]


In [87]:
def generate_features(sample_list, test = False):
    features = []
    i = 0
    for sample in sample_list:
        #print(sample)
        source = sample[0]
        target = sample[1]
        if test == False:
            label = sample[2]
        else:
            label = -1
        
        feature = []
        try:
            i = i+1
            #print(i)
            
            #p = nx.common_neighbors(g, source, target)
            #feature.append(len(p))
            
            #p = nx.simrank_similarity(g, source, target)
            #feature.append(p)
            
            #preds = nx.resource_allocation_index(g, [(source, target)])
            #for u, v, p in preds:
            #    feature.append(p)

            preds = nx.jaccard_coefficient(g, [(source, target)])
            for u, v, p in preds:
                feature.append(p)

            preds = nx.adamic_adar_index(g, [(source, target)])
            for u, v, p in preds:
                feature.append(p)

            #preds = nx.preferential_attachment(g, [(source, target)])
            #for u, v, p in preds:
            #    feature.append(p)
            
            feature.append(label)  # append label
            
        except Exception as e:
            #print(e)
            pass
        features.append(feature)
    print("features: "+str(len(features)))
    return features

In [88]:
features_pos = generate_features(edges_positive)
features_neg = generate_features(edges_negative)

features: 16087
features: 16087


In [89]:
features = features_pos + features_neg

In [91]:
def write_train_to_csv(features):
    with open("train_16k_sim.csv","w",newline="") as csvfile:
        writer=csv.writer(csvfile)
        writer.writerow(["JC","AA","Label"])
        writer.writerows(features)
        
write_train_to_csv(features)

In [92]:
dataset_sim = pd.read_csv('train_16k_sim.csv')
FEATURE_SIZE=2

X_sim = dataset_sim.iloc[:,:FEATURE_SIZE].values
y_sim = dataset_sim.iloc[:,FEATURE_SIZE].values

In [95]:
x_train_n2v_1 = np.array(x_train_n2v)

In [98]:
# Feature Scaling
sc = StandardScaler()
X_sim_1 = sc.fit_transform(X_sim)

In [100]:
all_feats = np.concatenate((X_sim_1,x_train_n2v_1),axis=1)

In [103]:
%%time
def objective(trial):
    train_x, valid_x, train_y, valid_y = train_test_split(all_feats, data['label'], 
                                                test_size = 0.2, 
                                                random_state = 35)
    dtrain = xgb.DMatrix(train_x, label=train_y)
    dvalid = xgb.DMatrix(valid_x, label=valid_y)

    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "n_estimators": trial.suggest_int("n_estimators", 0, 1000),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.005, 0.5),
        "nthread": -1,
        # use exact for small dataset.
        "tree_method": "exact",
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        # L2 regularization weight.
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
    }

    if param["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    bst = xgb.train(param, dtrain)
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    accuracy = accuracy_score(valid_y, pred_labels)
    return accuracy

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=300, timeout=600)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-04-07 14:22:42,587] A new study created in memory with name: no-name-f0b67ce3-52a3-4b3c-a897-1a0cd0aa5023
[I 2021-04-07 14:22:42,951] Trial 0 finished with value: 0.9504273504273504 and parameters: {'n_estimators': 580, 'learning_rate': 0.01077115841659831, 'booster': 'dart', 'reg_lambda': 5.9212808550032955e-08, 'alpha': 0.004091007062787086, 'subsample': 0.4222548094578822, 'colsample_bytree': 0.9654809755684464, 'max_depth': 9, 'min_child_weight': 4, 'eta': 0.2984297932009338, 'gamma': 4.645132783671683e-06, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 4.164933836636822e-06, 'skip_drop': 1.471597612907046e-05}. Best is trial 0 with value: 0.9504273504273504.
[I 2021-04-07 14:22:43,083] Trial 1 finished with value: 0.949028749028749 and parameters: {'n_estimators': 189, 'learning_rate': 0.017145201976796056, 'booster': 'gbtree', 'reg_lambda': 7.667834881776991e-06, 'alpha': 0.28710102277616006, 'subsample': 0.5167977020329084, 

Number of finished trials: 300
Best trial: {'n_estimators': 690, 'learning_rate': 0.3917114986734955, 'booster': 'gbtree', 'reg_lambda': 1.9040011449212134e-05, 'alpha': 2.594111166350005e-08, 'subsample': 0.997522324526165, 'colsample_bytree': 0.817500760413142, 'max_depth': 9, 'min_child_weight': 2, 'eta': 0.037409157001123794, 'gamma': 0.013127896580127005, 'grow_policy': 'lossguide'}
CPU times: user 16min 44s, sys: 1min 4s, total: 17min 48s
Wall time: 2min 21s


In [104]:
x_train, x_test, y_train, y_test = train_test_split(all_feats, data['label'], 
                                                test_size = 0.2, 
                                                random_state = 35)

In [105]:
clf = XGBClassifier(
    n_estimators= 690, 
    learning_rate= 0.3917114986734955, 
    booster= 'gbtree', 
    reg_lambda= 1.9040011449212134e-05, 
    alpha= 2.594111166350005e-08, 
    subsample= 0.997522324526165, 
    colsample_bytree= 0.817500760413142, 
    max_depth= 9, 
    min_child_weight= 2, 
    eta= 0.037409157001123794, 
    gamma= 0.013127896580127005, 
    grow_policy= 'lossguide')

In [106]:
clf.fit(x_train,y_train)

/usr/local/Caskroom/miniconda/base/envs/smlproj/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(alpha=2.594111166350005e-08, base_score=0.5, booster='gbtree',
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.817500760413142, eta=0.037409157001123794,
              gamma=0.013127896580127005, gpu_id=-1, grow_policy='lossguide',
              importance_type='gain', interaction_constraints='',
              learning_rate=0.3917114986734955, max_delta_step=0, max_depth=9,
              min_child_weight=2, missing=nan, monotone_constraints='()',
              n_estimators=690, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=2.59411124e-08, reg_lambda=1.9040011449212134e-05,
              scale_pos_weight=1, subsample=0.997522324526165,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [107]:
pre=clf.predict_proba(x_test)
# print(pre)
y_pre=[p[1] for p in pre]
acc=clf.score(x_test,y_test)
print(acc)
auc=roc_auc_score(y_test,y_pre)
print(auc)

0.9613053613053613
0.9891986420951283


In [108]:
testCols=['Id', 'source_node', 'destination_node'] 
df_test_public = pd.read_csv('test-public.csv')
ids = df_test_public['Id'].values
df_test_public.columns = testCols
df_test_public = df_test_public.drop('Id', axis = 1)
df_test_public

,source_node,destination_node
0,0,2917
1,0,2956
2,1,4038
3,2,1848
4,3,513
...,...,...
1995,3865,3924
1996,3917,4025
1997,3922,3947
1998,3955,3987


In [109]:
x_testing_n2v = [(n2w_model[str(i)]+n2w_model[str(j)]) for i,j in zip(df_test_public['source_node'], df_test_public['destination_node'])]

<ipython-input-109-2271668228b8>:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  x_testing_n2v = [(n2w_model[str(i)]+n2w_model[str(j)]) for i,j in zip(df_test_public['source_node'], df_test_public['destination_node'])]


In [111]:
x_testing_n2v_1 = np.array(x_testing_n2v)

In [113]:
test_edges = df_test_public.to_numpy()

In [114]:
features_test = generate_features(test_edges, test=True)

features: 2000


In [115]:
def write_test_to_csv(features):
    with open("test.csv","w",newline="") as csvfile:
        writer=csv.writer(csvfile)
        writer.writerow(["JC","AA","Label"])
        writer.writerows(features)

In [116]:
write_test_to_csv(features_test)

In [117]:
dataset_test_sim = pd.read_csv('test.csv')
FEATURE_SIZE=2

X_test_sim = dataset_test_sim.iloc[:,:FEATURE_SIZE].values
y_test_sim = dataset_test_sim.iloc[:,FEATURE_SIZE].values

In [118]:
x_testing_sim = sc.transform(X_test_sim)

In [120]:
all_feats_testing = np.concatenate((x_testing_sim,x_testing_n2v_1),axis=1)

In [122]:
predictions=clf.predict_proba(all_feats_testing)

In [124]:
with open("XGBoost_results.csv","w",newline="") as csvfile:
    writer=csv.writer(csvfile)
    writer.writerow(["Id","Predicted"])
    test_id=1
    for prediction in predictions:
        writer.writerow([test_id,prediction[1]])
        test_id+=1